In [14]:
import requests
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [15]:
def get_parcel_data(bpoly):
    # Get the coordinates of the bounding box for input polygon bpoly
    bbox = bpoly.bounds
    
    # Construct the query URL to get the parcel data within the bounding box
    # Will change the following California URL to universal code to extract the corresponding URL for different state from the spreadsheet
    query = ('https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/' +
             'rest/services/CA_Statewide_Parcels_Public_view/FeatureServer/0/query?' +
             f'geometry={bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}' +
             '&geometryType=esriGeometryEnvelope&inSR=4326' + 
             '&spatialRel=esriSpatialRelIntersects' +
             '&outFields=*' +
             '&returnGeometry=true&f=json')
    
    # Set up a session with retries
    s = requests.Session()
    retries = Retry(total=5, 
                    backoff_factor=0.1,
                    status_forcelist=[500, 502, 503, 504])
    s.mount('https://', HTTPAdapter(max_retries=retries))

    # Send the request and get the response
    r = s.get(query)
    
    # Get the parcel data in JSON format
    parcel_data = r.json()
    
    return parcel_data

# Alameda Island Example

In [16]:
from shapely.geometry import box
# Define the bounding box for Alameda Island
alameda_bbox = box(-122.3164, 37.7393, -122.2307, 37.7957)

# Get the parcel data for Alameda Island in JSON format
alameda_parcel_data = get_parcel_data(alameda_bbox)

In [17]:
# Function to convert data from json to Pandas DataFrame
def json_to_dataframe(json_data):
    # Extract the features from the JSON data
    features = json_data['features']
    # Extract the attributes for each feature
    data = [feature['attributes'] for feature in features]
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    # Return the DataFrame
    return df

In [18]:
# Convert the Alameda Island parcel data to the DataFrame
alameda_parcel_df = json_to_dataframe(alameda_parcel_data)

alameda_parcel_df.head()

,OBJECTID,PARCEL_APN,FIPS_CODE,PARCEL_DMP_ID,COUNTYNAME,SITE_ADDR,SITE_CITY,SITE_STATE,SITE_ZIP,SITE_PLUS_4,...,SITE_UNIT_NUMBER,SITE_HOUSE_NUMBER,SITE_DIRECTION,SITE_STREET_NAME,SITE_MODE,SITE_QUADRANT,FullStreetAddress,Search_PARCELAPN,Shape__Area,Shape__Length
0,12651225,69-128-20,06001,100660192_173255035,ALAMEDA,1734 CAMBRIDGE DR,ALAMEDA,CA,94501,1614,...,,1734,,CAMBRIDGE,DR,,"1734 CAMBRIDGE DR, ALAMEDA, CA 94501",ALAMEDA 69-128-20,4.516232e-08,0.000933
1,12615676,21-259-11,06001,511898557_5896,ALAMEDA,2006 14TH AVE,OAKLAND,CA,94606,4006,...,,2006,,14TH,AVE,,"2006 14TH AVE, OAKLAND, CA 94606",ALAMEDA 21-259-11,2.262050e-08,0.000647
2,12632662,74-1355-64,06001,100660192_173238962,ALAMEDA,410 HOLLISTER AVE,ALAMEDA,CA,94501,3182,...,,410,,HOLLISTER,AVE,,"410 HOLLISTER AVE, ALAMEDA, CA 94501",ALAMEDA 74-1355-64,3.066452e-08,0.000691
3,12633249,74-430-26,06001,100660192_173239474,ALAMEDA,610 PACIFIC AVE,ALAMEDA,CA,94501,2127,...,,610,,PACIFIC,AVE,,"610 PACIFIC AVE, ALAMEDA, CA 94501",ALAMEDA 74-430-26,4.867434e-08,0.000970
4,12633913,74-1351-58,06001,100660192_173240177,ALAMEDA,5 ELLIS CT,ALAMEDA,CA,94501,6405,...,,5,,ELLIS,CT,,"5 ELLIS CT, ALAMEDA, CA 94501",ALAMEDA 74-1351-58,2.539264e-08,0.000649
